In [41]:
from sympy.matrices import Matrix
from sympy import pprint, Rational as rat, solve, symbols

Для системы линейных уравнений $Ax = f$ из предыдущей задачи привести вычислительные формулы и выполнить три итерации методов Якоби, Зейделя и верхней релаксации, выбрав итерационный параметр, близкий к оптимальному. Провести три шага вычислений для определения максимального по модулю собственного значения матрицы системы степенным методом, взяв в качестве начального приближения вектор $x$.

Матрица А и вектор f:

In [67]:
A = Matrix([[rat(3, 1), rat(1, 1)], [rat(8, 1), rat(5, 1)]])
f = Matrix([[rat(2, 1)], [rat(3, 1)]])
f

Matrix([
[2],
[3]])

In [25]:
x, y = symbols('x,y')
first_row = A.row(0) * Matrix([x, y])
second_row = A.row(1) * Matrix([x, y])
eq1 = first_row - f.row(0)
sol1 = solve(eq1, x)
sol1

{x: 2/3 - y/3}

In [26]:
eq2 = second_row - f.row(1)
sol2 = solve(eq2, y)
sol2

{y: 3/5 - 8*x/5}

Возьмём в качестве начального вектора нулевой и проведём три итерации разными методами:

In [23]:
x_0 = 0
y_0 = 0

Якоби:


In [39]:
x_1, x_2, y_1 = x_0, x_0, y_0
for _ in range(3):
    x_1 = solve(eq1.subs({'y' : y_1}), x)[x]
    y_1 = solve(eq2.subs({'x' : x_2}), y)[y]
    x_2 = x_1
    pprint((x_1, y_1))

(2/3, 3/5)
(7/15, -7/15)
⎛37  -11 ⎞
⎜──, ────⎟
⎝45   75 ⎠


Зейдель:

In [40]:
x_1, y_1,= x_0, y_0
for _ in range(3):
    x_1 = solve(eq1.subs({'y' : y_1}), x)[x]
    y_1 = solve(eq2.subs({'x' : x_1}), y)[y]
    pprint((x_1, y_1))

(2/3, -7/15)
⎛37  -161 ⎞
⎜──, ─────⎟
⎝45   225 ⎠
⎛611  -2863 ⎞
⎜───, ──────⎟
⎝675   3375 ⎠


Система для ПВР:
$$
x^{k+1} = (1 - \omega) x^k + \frac{\omega}{3} (2 - y^k), \\
y^{k+1} = (1 - \omega) y^k + \frac{\omega}{5} (3 - 8y^k),
$$

Теперь найдём оптимальное значение $\omega = \frac{2}{1 + \sqrt{1 - \lambda ^ 2}}$, где $\lambda$ -- собственное число матрицы:

In [45]:
R = Matrix([[rat(0, 1), rat(-1, 3)], [rat(-8, 5), rat(0, 1)]])
R.eigenvals()

{-2*sqrt(30)/15: 1, 2*sqrt(30)/15: 1}

In [48]:
lambda_2 = 1
for val in R.eigenvals().keys():
    lambda_2 *= val
lambda_2 = abs(lambda_2)
lambda_2

8/15

In [65]:
w = round(2 / (1 + (1 - lambda_2) ** 0.5), 3)
w

1.188

In [66]:
x_new, y_new = symbols('x_new,y_new')
eq3 = x_new - ((1 - w) * x + w / 3 * (2 - y))
eq4 = y_new - ((1 - w) * y + w / 5 * (3 - 8 * x))

x_1, y_1,= x_0, y_0
for _ in range(3):
    x_1 = round(solve(eq3.subs({'x': x_1, 'y': y_1}), x_new)[0], 3)
    y_1 = round(solve(eq4.subs({'x' : x_1, 'y': y_1}), y_new)[0], 3)
    pprint((x_1, y_1))

(0.792, -0.793)
(0.957, -0.957)
(1.0, -1.008)


Теперь найдём максимальное собственное число по методу Вовы:
$r_{k+1} = \frac{A r_k}{||r_k||}$ - вектор с наибольшим собственным числом, затем получим само число $\lambda_{max} = \frac{r^T A r}{r^T r}$.

In [73]:
r = Matrix([rat(1, 1), rat(1, 1)])
for _ in range(3):
    r = (A * r / (r.dot(r)) ** 0.5).evalf(3)
    pprint(r)
    print()

⎡2.83⎤
⎢    ⎥
⎣9.19⎦

⎡1.84⎤
⎢    ⎥
⎣7.13⎦

⎡1.72⎤
⎢    ⎥
⎣6.84⎦



In [76]:
l_max = r.T * A * r / (r.dot(r))
l_max

Matrix([[7.01]])